In [1]:
# Importing necessary modules
import pandas as pd

In [2]:
# Reading in the answers file, keeping only awards and nominees
answers = pd.read_csv('answers.csv', usecols = ['award', 'nominees'])
answers

,award,nominees
0,best screenplay - motion picture,"[zero dark thirty, lincoln, silver linings pla..."
1,best director - motion picture,"[kathryn bigelow, ang lee, steven spielberg, q..."
2,best performance by an actress in a television...,"[zooey deschanel, tina fey, julia louis-dreyfu..."
3,best foreign language film,"[the intouchables, kon tiki, a royal affair, r..."
4,best performance by an actor in a supporting r...,"[alan arkin, leonardo dicaprio, philip seymour..."
5,best performance by an actress in a supporting...,"[hayden panettiere, archie panjabi, sarah paul..."
6,best motion picture - comedy or musical,"[the best exotic marigold hotel, moonrise king..."
7,best performance by an actress in a motion pic...,"[emily blunt, judi dench, maggie smith, meryl ..."
8,best mini-series or motion picture made for te...,"[the girl, hatfields & mccoys, the hour, polit..."
9,best original score - motion picture,"[argo, anna karenina, cloud atlas, lincoln, li..."


In [ ]:
# Reading in and processing the gg2013 tweets file